<a href="https://colab.research.google.com/github/ozmdm/ADSS_Group_M/blob/master/Latent_constructs_in_NLP_test_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

### Submission details

Fill in your details

In [1]:
#!export
#replace the following value with a hardcoded ingeger of your ID (all digits without leading zeroes)
student_id = 311529721

#replace the following value with the number of questions you need to translate.
n_questions = 10

### Miscelaneous

This section includes miscleaneous code for enabling questionnaire translation. **Please do not change** this code.

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
print(IN_COLAB)

True


In [3]:
if IN_COLAB:
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
if IN_COLAB:
  from psutil import virtual_memory
  ram_gb = virtual_memory().total / 1e9
  print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

  if ram_gb < 20:
    print('Not using a high-RAM runtime')
  else:
    print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [5]:
if IN_COLAB:
    !pip install transformers
    !pip install sentencepiece
    !pip install datasets
    !pip install ray
    !pip install overrides
    !pip install accelerate
    !pip install sentence-transformers
    !pip install typeguard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

In [6]:
# !wget "https://drive.google.com/uc?id=1M7M7amV0x0S038BgevEybczbm3buCWuX&export=download" -O qmnli.pyc

In [7]:
#!export
import torch
import pandas as pd
import numpy as np
import torch
import time
from pprint import pprint
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import pipeline
from transformers import PreTrainedModel
from transformers import PreTrainedTokenizer
import scipy
import sklearn as sk
import ray
from transformers import GPTJForCausalLM, AutoTokenizer
from pathlib import Path
import itertools
import inspect
from sentence_transformers import SentenceTransformer, util
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

device = 0 if torch.cuda.is_available() else -1
print(device)

-1


In [8]:
#!export
import torch
import pandas as pd
import torch
import pandas as pd
import numpy as np
import sklearn as sk
import torch
from pprint import pprint
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import pipeline
from transformers import PreTrainedModel
from transformers import PreTrainedTokenizer
import scipy
import time
import itertools
import sys
import warnings

from functools import partial
# from qabstract.qabstract import QABSTRACT
from overrides import override
from abc import *
import copy

from string import Formatter

from typing import *
from typeguard import check_type
from numbers import Number

SCALE = Dict[str,Number]
DIMENSIONS = Dict[str,SCALE]
FILTER = Dict[str,Collection[str]]
IDXSELECT = Tuple[Union[slice,List[int]]]

## Wait for colab to upgrade to Python 3.11
##IDXSELECT_for_consistency_checks = Annotated[Tuple[Union[slice,Annotated[List[int], MinLen(2)]], MinLen(2)]

def fixed_check_type(var, expected_type):
  try:
    check_type(var, expected_type)
    return True
  except:
    return False

def _filter_tensor(t, slices):
      for i in range(len(slices)):
        t = t[slices[i]]
      return t

def _filter_data_frame(df, filter:FILTER):
      select = df.copy()
      for f in filter: #only select rows where df[f] in filter[f]
          select[f] = select[f].apply(lambda x: x in filter[f])
      select = select.all(axis=1)
      return select

def print_gradient(df):
  import seaborn as sns
  cm = sns.light_palette("green", as_cmap=True)
  s = df.style.background_gradient(cmap=cm, axis=None)
  s = s.set_precision(4)
  return s


def wrap_replace_callable(c, f, with_copy=False, *fargs, **fkwargs):
    def wrapper(*cargs, **ckwargs):
        rc = c(*cargs, **ckwargs)
        rf = f(rc, *fargs, **fkwargs)
        return rf
    if with_copy:
        q = copy.deepcopy(c)
    else:
        q = c
    q.__call__ = wrapper
    return q


class QMNLI:

    # Static property QMNLI._qregister is defined after this class
    # It will contain one instance (the last one) of every subtype of QMNLI.
    # This will allow to automatically register questions and reuse them with various models when the time comes.

    def __init__(self,
                 premise_template: str = " ",
                 hypothesis_template: str = " ",
                 dimensions:DIMENSIONS = {},
                 model: pipeline = None,
                 p=None,
                 index=None,
                 scale='intensifier',
                 descriptor = {}):
      """
      Templates (both context and answer) are parsed before execution to prepare the list of template fields used in each string.
      During execution a list of strings and a list of answer strings are created basaed on cartesian product of the respective field values.
      The field values are extracted from dimensions and intensifiers.
      """
      self._dimensions = dimensions
      self._field_names = list(self._dimensions.keys())
      #print(f"_field_names={self._field_names}")
      self._scale = scale
      self.result = None
      self._index = index if index is not None else list(set(self._field_names) - set([scale]))
      self._descriptor = descriptor
      self._descriptor['scale'] = str(self._scale)
      self._descriptor['index'] = str(self._index)
      self._descriptor['query'] = premise_template+"->"+hypothesis_template


      self._keywords = {d:list(scale.keys()) for d,scale in dimensions.items()}
      #print(f"_keywords={self._keywords}")
      self._keywords_indices = {d:dict([(k,i) for i,k in enumerate(self._keywords[d])]) for d in self._keywords}
      #print(f"_keywords_indices={self._keywords_indices}")
      self._dimshape = tuple([len(self._keywords[d]) for d in self._field_names])
      #print(f"_dimshape={self._dimshape}")

      self._keywords_grid = list(itertools.product(*[self._keywords[f] for f in self._field_names]))
      #print(f"_keywords_grid={self._keywords_grid}")
      self._keywords_map = [dict(zip(self._field_names,k)) for k in self._keywords_grid]
      #print(f"_keywords_map={self._keywords_map}")
      self._keywords_grid_idx = torch.Tensor([tuple([self._keywords_indices[f][ktuple[i]] for i,f in enumerate(self._field_names)]) for ktuple in self._keywords_grid])
      #print(f"_keywords_grid_idx={self._keywords_grid_idx[:3]}")


      W = [tuple([self._dimensions[f][ktuple[i]] for i,f in enumerate(self._field_names)]) for ktuple in self._keywords_grid]
      #print(f"W={W}")
      self._weights_grid = pd.DataFrame(W, columns=self._field_names)
      #print(f"_weights_grid={self._weights_grid}")
      self._weights_flat = self._weights_grid.prod(axis=1)
      #print(f"_weights_flat={self._weights_flat}")
      self._weights_dense = torch.sparse_coo_tensor(self._keywords_grid_idx.T, self._weights_flat, self._dimshape).to_dense()
      #print(f"_weights={self._weights.shape}")


      self._pdf = pd.DataFrame(self._keywords_grid, columns=self._field_names)
      self._pdf = self._pdf.assign(P=0)
      self._pdf = self._pdf.assign(W=self._weights_flat)
      self.class_name = self.__class__.__name__
      #print(f"_pdf={self._pdf}")



      self._context_template = premise_template

      self._answer_template = hypothesis_template

      self._p = p
      self.model = model

      QMNLI._qregister[self.__class__.__name__]=self

    def __call__(self, model=None):
        return self.run(model)

    def set_model(self, model):
      self.model = model

    def get_class_name(self):
      return self.class_name

    def run(self, model=None):
        if model:
            self.model = model
        T = time.time()
        coo = []
        p = []
        for kmap,kcoo in zip(self._keywords_map,self._keywords_grid_idx):
          context = self._context_template.format_map(kmap)
          answer = self._answer_template.format_map(kmap)
          ans = self.model(
                sequences=context,
                hypothesis_template="{}",
                multi_label=True,
                candidate_labels=answer)
          #single sequence single label
          p.append(torch.Tensor([ans["scores"][0]]).squeeze().cpu().item())
          coo.append(kcoo)

        coo = torch.stack(coo).T
        assert torch.all(torch.eq(coo.T, self._keywords_grid_idx))

        self._pdf["P"] = p

        self._T = time.time() - T
        self.result = self
        return self.result

    def _grouping_suitable_for_consistency_check(self, grouping, filter={}):
      if len(grouping)<2:
        return False
      label_count = 0
      for group_filter in grouping:
        df = self._pdf[_filter_data_frame(self._pdf, filter)]
        df = df[_filter_data_frame(df,group_filter)]
        if len(df) > 0:
          label_count += 1
      return label_count >= 2

    def report(self,scale:Union[str,int]=None, index:List[str]=None, filters:Dict[str,FILTER]={"unfiltered":{}}, grouping:List[FILTER]=[],):
        scale = self._scale if scale is None else scale
        if not fixed_check_type(scale, str):
          scale = self._field_names.index(scale)
        if index is None:
            if self._index:
                index = self._index
            else:
                index = list(set(self._field_names) - set([scale, "P", "W"]))

        print(f"Query time: {self._T}")
        for label, filter_dict in filters.items():
          print(f"Mean score {label}: {self.mean_score(filter=filter_dict)}")

         # Add default grouping for the index field
        if grouping is not None and len(grouping) == 0:
          group_field = index[0]
          grouping = self._create_default_grouping(group_field)

        for label, filter_dict in filters.items():
          if self._grouping_suitable_for_consistency_check(grouping, filter=filter_dict):
            partial_internal_consistency = partial(self.internal_consistency, grouping=grouping,filter=filter_dict, index=index , scale=scale)
            print(f"Internal consistency (silhouette, correlation) for {label}: {partial_internal_consistency(measure='silhouette_score', metric='correlation')}")
            print(f"Internal consistency (Calinski&Harabasz)  for {label}: {partial_internal_consistency(measure='calinski_harabasz_score')}")
            print(f"Internal consistency (Davies&Bouldin) for {label}: {partial_internal_consistency(measure='davies_bouldin_score')}")
          else:
            print("At least two groups with at least two vectors in each group should be specified to check for internal consistency.")

        for label, filter_dict in filters.items():
            print("\n")
            display(
              print_gradient(
                  self.to_dataframe(
                    scale=scale,
                    index=index,
                    filter=filter_dict,
            )))


    def _filter_words_to_slice(self, filter:FILTER)->IDXSELECT:
      s = [ #loop over dimensions
                        slice(None,None) if d not in filter else
                        [ #loop over keywords in dimesion
                            self._keywords_indices[d][k]
                            for k in filter[d]
                        ]
                        for d in self._field_names
                      ]
      #one-hot econding, otherwise the shapes won't broadcast together...
      n = len(self._field_names)
      result = [[slice(None,None,None) for _ in range(n)] for _ in range(n)]
      for i in range(n):
        result[i][i]=s[i]
      return result


    def mean_score(self, filter:FILTER={}):
      select = _filter_data_frame(self._pdf,filter)
      P = self._pdf[select]
      score = P["P"]*P["W"]
      score = score.sum()
      return score


    def _create_default_grouping(self, group_field):
      default_grouping_df = pd.DataFrame(self._dimensions[group_field].items(), columns=['name', 'score'])
      grouping = [{group_field: group_df['name'].tolist()} for group_score, group_df in default_grouping_df.groupby('score')]
      # print(f'No grouping stated, using default grouping on {group_field}:')
      # pprint(grouping)
      return grouping

    def internal_consistency(self, measure="silhouette_score", metric="correlation", grouping:List[FILTER]=[], scale:Union[str,int] = None, index:List[str] = None, filter:FILTER={}):
        """
        Before the internal consistency check the data is diced according to the filter -- e.g. only entries matching the filter will be considered.
        This operation first splits the data into scale long vectors. The vectors are grouped according to the grouping argument.
        The vectors within each group are expected to be similar (high correlation, low distance) while vectors in different groups are expected to be different.
        This expectation is validated by Scikit's unsupervised cluster quality measures.

        Parameters:
        -----------
        @filter
            Specifies a set of keywords to keep along chosen dimensions. Unspecified dimensions are kept (not filtered out).
        @scale
            Dimension specifying the vector for which distances are computed. This is the dimension which is used for likert scale values.
        @grouping
            Specifies multiple dices (each one is structured like the filter) within which likert scale values are expected to be the same.
            Likert scale values are expected to be different for different dices. Dices are not required to have the same shape.
        @measure: string
            An unsupervised cluster quality measure from scikit. Must be one of 'calinski_harabasz_score', 'davies_bouldin_score', 'silhouette_score', 'silhouette_samples'
        @metric: string
            One of distance metrics from 'sklearn.metrics.pairwise_distances' used for computing silhouette
        """
        try:
          scale = self._scale if scale is None else scale
          check_type(scale, int)
          scale = self._field_names[scale]
        except:
          check_type(scale, str) #redundant check for clarity

        # Add default grouping for the index field
        index = self._index if index is None else index
        if grouping is not None and len(grouping) == 0:
          group_field = index[0]
          grouping = self._create_default_grouping(group_field)

        df = self._pdf[_filter_data_frame(self._pdf, filter)]

        X = []
        L = []
        for label,group_filter in enumerate(grouping):
          dice = df[_filter_data_frame(df, group_filter)]
          dice = dice.pivot_table(index=index,columns=scale, values="P", aggfunc='mean')
          X += [dice]
          L += [label]*len(dice)
        X = pd.concat(X, axis=0)

        f = getattr(sk.metrics, measure)
        if 'metric' in f.__code__.co_varnames[:f.__code__.co_argcount + f.__code__.co_kwonlyargcount]:
            result = f(X=X, labels=L, metric=metric)
        else:
            result = f(X=X, labels=L)

        return result

    def _pd_index_sort_key(self, terms:pd.Index)->pd.Index:
      f = terms.name
      result = pd.Index([self._dimensions[f][x] for x in terms])
      return result

    def _pd_values_sort_key(self, terms:pd.Series)->pd.Series:
      f = terms.name
      result = terms.apply(lambda x: self._dimensions[f][x])
      return result


    def to_dataframe(self, scale:Union[str,int], index:List[str], filter:FILTER={}, categories:Dict[str,Dict[str,str]]={}):
        try:
          check_type(scale, int)
          scale = self._field_names[scale]
        except:
          check_type(scale, str) #redundant check for clarity

        select = _filter_data_frame(self._pdf, filter)
        df = self._pdf[select]
        print(f"index = {index}")
        print(set(self._field_names), set([scale]), set(index) )
        aggregated_fields = set(self._field_names) - set([scale]) - set(index)
        print(aggregated_fields)
        if len(aggregated_fields)>0:
            W = self._weights_grid[select]
            W = W[aggregated_fields]
            W = W.prod(axis=1)
            df["P"] = df["P"]*W


        for f in categories.items(): #replace individual keywords with categories
          df[f] = df[f].apply(lambda x: categories[f][x])



        #group by index categories
        df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)
        df = df.sort_index(axis=1, key=self._pd_index_sort_key)
        df = df.sort_index(axis=0, key=self._pd_index_sort_key)

        return df

    def softmax(self, dim=Union[str,int,List[str],List[int]]):
        """
        params:
        dim: int or tuple. The dimensions to normalize with softmax. Dimesions will be normalized one by one in the given order.
        """
        if not fixed_check_type(dim, List):
          dim = [dim]
        if fixed_check_type(dim, List[str]):
          dim = [self._field_names.index(d) for d in dim]
        elif fixed_check_type(dim, List[int]):
          pass
        else:
          raise TypeError

        result = copy.deepcopy(self,memo={id(self.model):self.model})
        # del result.model
        # result.model = self.model
        coo = self._keywords_grid_idx.T
        p = torch.Tensor(self._pdf["P"])
        p_dense = torch.sparse_coo_tensor(coo, p, self._dimshape).to_dense()

        p_sparse = p_dense.to_sparse()
        p_vals = p_sparse.values()
        p_coos = p_sparse.indices()
        assert torch.all(torch.eq(p, p_vals))
        assert torch.all(torch.eq(coo, p_coos))

        for d in dim:
            p_dense = torch.nn.functional.softmax(p_dense, dim=d)

        p_vals = p_dense.to_sparse().values()

        result._pdf["P"] = p_vals
        return result

    # # def __str__(self):
    # #   return f"<{self.__class__.__name__} object> {self._descriptor}"
    # def __repr__(self):
    #   return f"<{self.__class__.__name__} object> {self._descriptor}"
    def __hash__(self):
      """
      Questions having the same are descriptor should represent the same question.
      """
      return hash(frozenset(self._descriptor.items()))
    def __eq__(self,other):
      """
      Questions having the same are descriptor represent the same question.
      """
      return self._descriptor == other._descriptor



#     def __add__(self,other):
#         result = copy.deepcopy(self)
#         result._p += other._p
#         return result
#     def __sub__(self,other):
#         result = copy.deepcopy(self)
#         result._p -= other._p
#         return result
#     def __mul__(self,other):
#         result = copy.deepcopy(self)
#         result._p *= other._p
#         return result
#     def __truediv__(self,other):
#         result = copy.deepcopy(self)
#         result._p /= other._p
#         return result

# The static property QMNLI._qregister will contain one instance (the last one) of every subtype of QMNLI.
# This will allow to automatically registering questions and reusing them with various models when the time comes.

QMNLI._qregister:Dict[str,QMNLI]={}


class _QMNLI(QMNLI):
  def __init__(self, context, template, emo_pos, emo_neg, intensifiers):
    super().__init__(
        context_template=context,
        answer_template=template,
        dimensions={"emotion":dict(zip(emo_pos+emo_neg,[1]*len(emo_pos)+[-1]*len(emo_neg))), "intensifier":intensifiers, }
    )
    self.scale = "intensifier"
    self.index = "emotion"
    self.emo_pos = emo_pos
    self.emo_neg = emo_neg
    self.grouping = [{"emotion":emo_pos},{"emotion":emo_neg}]

  def report(self,scale:Union[str,int]="intensifier", index="emotion", filters:Dict[str,FILTER]={"unfiltered":{}}, grouping:List[FILTER]=[],):
    if len(grouping)==0:
      grouping = self.grouping
    return super().report(scale=scale,index=index,filters=filters,grouping=grouping)

  def softmax(self, dim:Union[str,int,List[str],List[int]]="intensifier"):
    return super().softmax(dim=dim)

  def to_dataframe(self, scale:Union[str,int]="intensifier", index:List[str]="emotion", filter:FILTER={}, categories:Dict[str,Dict[str,str]]={}):
    return super().to_dataframe(scale=scale, index=index, filter=filter, categories=categories )


class QDELEGATOR(QMNLI):
    def __init__(self, srcobj):
      """
      Shallow copy the state of the source object.
      Both objects will share the same current state.
      The run method of QDELEGATOR delegates the execution to srcobj.run and
      copies the state after each execution. Any modifications made to the state
      of srcobj prior to execution of run(..) are copied as well.
      Further modifications to the shallow state of either one of the objects will
      not affect the other one.
      @srcobj
          initialized object of a question.
      """
      super().__init__()
      self.__dict__.update(srcobj.__dict__)
      self._run = srcobj.run

    @override
    def run(self, model=None, **kwargs):
      print(f"{self.__class__.__name__} delgates execution of run(..) to {self._run.__self__.__class__.__name__}" )
      result = self._run(model, **kwargs)
      d = dict(result.__dict__)
      #retain self._descriptor and self._run
      if "_descriptor" in d: del d['_descriptor']
      if "_run" in d: del d['_run']
      self.__dict__.update(d)
      # #restore self._run to maintain full delegation path
      # self._run = result.run
      return self

class QCACHE(QDELEGATOR):
    def __init__(self, srcobj):
      super().__init__(srcobj)
      #subclasses instantiated with questions that were already run
      #may set the cached flag to False to force rerunning the question
      self._cached = False
      self._last_model_identifier = None

    @override
    def run(self,model=None, **kwargs):
      # print(f"Execute cachable question with model {model}.")
      # print(f"Cached: {self._cached}, same model:{model.model_identifier == self._last_model_identifier}.")
      if self._cached and model.model_identifier == self._last_model_identifier:
        # print(f"Skipping model execution. Use cached results.")
        return self
      result = super().run(model=model,**kwargs)
      self._cached = True
      self._last_model_identifier = model.model_identifier
      assert result==self
      return self



class QPASS(QDELEGATOR):
    def __init__(self, qobject, descupdate={}):
        super().__init__(qobject)
        #copy descriptor from srcobj to avoid propagation of changes
        self._descriptor = dict(self._descriptor, **descupdate)


class QSOFTMAX(QPASS):
    def __init__(self, qobject, dim="intensifier"):
        super().__init__(qobject, {'softmax':str(dim)})
        self._dim = dim

    @override
    def run(self,model=None, **kwargs):
        result = super().run(model, **kwargs)
        return result.softmax(dim=self._dim)


class QFILTER(QPASS):
    def __init__(self, qobject, filter:FILTER = {}, filtername=""):
        super().__init__(qobject, {'filter':filtername})
        self._filter = filter

    @override
    def run(self,model=None, **kwargs):
        result = super().run(model, **kwargs)
        assert result == self
        select = _filter_data_frame(self._pdf, self._filter)
        result._pdf = self._pdf[select]
        result._weights_grid = self._weights_grid[select]
        return result



### Testing code

This is the code that is used to tests your assignments do not change it

In [9]:
sentence_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
cola = pipeline("text-classification","mrm8488/deberta-v3-small-finetuned-cola", device=device)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [10]:
import os
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
import string

def clean_text(text):
  if isinstance(text,str):
    a_string =  text
    new_string = a_string.translate(str.maketrans('', '', '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~' + '0123456789'))
    return ' '.join(new_string.split())
  else:
    return ''

def linguistic_acceptabilities(q, question_id, index, scale,question_name, student_id, output_path=Path('')):
    score_by_cola_lst=[]
    score_of_semantic_distance_lst=[]
    score_by_bleu_lst=[]
    kmap_lst=[]
    question_name_lst=[]
    description = q._descriptor
    strFactor=description['Factor']
    strOrdinal=str(description.get('Ordinal', 0))
    ##cleaning the string to get the original question
    strOriginal= description['Original']
    strOriginal = 'none' if strOriginal is None else strOriginal
    strOriginal=strOriginal.replace(strFactor,'',1)
    strOriginal=strOriginal.replace(strOrdinal,'',1)
    strOriginal=strOriginal.replace('.','',1)
    strOriginal=strOriginal.strip() #the original question

    if question_id != -1:
      context = clean_text(all_questionnairs.loc[question_id]['Context'])
      contex_terms =  clean_text(all_questionnairs.loc[question_id]['Context\Term'])
      answers = clean_text(all_questionnairs.loc[question_id]['Intensifire'])
      goal_sentence = f'{context} {contex_terms} {answers}'
    else:
      goal_sentence = strOriginal
    rows = []
    q = q.run(mnli)
    # scale = max(q._field_names, key=lambda x: np.max(list(q._dimensions[x].values())) > 1)
    # index = list(set(q._index) - {scale})
    # index = [x for x in index if np.max(list(q._dimensions[x].values())) == 1 and np.min(list(q._dimensions[x].values())) == -1]
    # group_field = sorted(index, key=lambda x: len(q._dimensions[x]), reverse=True)[0]
    # grouping = q._create_default_grouping(group_field)
    partial_internal_consistency = partial(q.internal_consistency, filter={}, index=index , scale=scale)
    try:
        silhouette_score = partial_internal_consistency(measure='silhouette_score', metric='correlation')
    except Exception as e:
        print(e)
        print('silhouette_score is set to -1')
        silhouette_score = -1

    for kmap in q._keywords_map:
        score = {}
        score['question_name'] = question_name
        context = q._context_template.format_map(kmap)
        answer = q._answer_template.format_map(kmap)
        score['original_question'] = goal_sentence


        cola_score = cola(context +" "+ answer)[0].get('score')
        score['cola_score'] = cola_score
        score['param'] = kmap
        strPermutation= context +" "+ answer
        # sentences = [context +" "+ answer]
        score['question_permutation'] = strPermutation
        #Compute embedding for both lists
        embeddings1 = sentence_embedding_model.encode(goal_sentence, convert_to_tensor=True)
        embeddings2 = sentence_embedding_model.encode(strPermutation, convert_to_tensor=True)

        #Compute cosine-similarities
        cosine_scores = util.cos_sim(embeddings1, embeddings2)
        score['semantic_similarity'] = cosine_scores.item()

        score['silhouette_score'] = silhouette_score
        rows.append(score)


    filename = output_path / 'linguistic_acceptabilities.csv'
    df = pd.DataFrame(rows)
    df['student_id'] = student_id
    df = df[['student_id', 'question_name','original_question', 'param','question_permutation','cola_score','semantic_similarity','silhouette_score']]
    if filename.exists():
        df.to_csv(filename, index=False, header=None, mode='a', encoding='utf-8-sig')
    else:
        df.to_csv(filename, index=False, encoding='utf-8-sig')
    return df

In [11]:
def check_question_not_default(q, question_name):
  def_descriptor = {"Questionnair":"QuestionnairName",
                      "Factor":"FactorName",
                      "Ordinal":999,
                      "Original":'the original question',
        }
  def_hypothesis_template="hypothesis {scale} ",
  result = True
  if 'QuestionnairName' in question_name:
      print('\tplease update QuestionnairName')
      result = False
  if 'QuestionNumber' in question_name:
      print('\tplease update QuestionNumber')
      result = False
  for key, val in def_descriptor.items():
      if val == q._descriptor[key]:
          print(f'\tplease update key: <{key}> in the descriptor.')
          result = False

  if q._context_template == "premise {index}":
      print('\tplease update premise')
      result = False
  if q._answer_template == "hypothesis {scale}":
      print('\tplease update hypothesis')
      result = False

  return result


In [12]:
def print_permutations(q):
#     for q in Q1s:
    W = q._pdf['W']
    print(q._descriptor)
    for i, (kmap, w) in enumerate(zip(q._keywords_map, W)):
        context = q._context_template.format_map(kmap)
        answer = q._answer_template.format_map(kmap)
#         sexisem_score = sexisem_classifier(context.strip('.') + ' ' +answer)
        print(f'{i}.',context ,'->', answer, w)

### Usefull dictionaries and functions

`print_gradient` function displays a pandas DataFrame with entries colored according to their values. It is usefull for visualizing the model responses and understanding the quality of the question.

`mnli` variable holds a small MNLI pipeline  for testing and debugging the questions.   

Various discionaries contain sets of intensifiers with their respective wieights.

In [13]:
# Please do not change this function
def print_gradient(df):
  import seaborn as sns
  cm = sns.light_palette("green", as_cmap=True)
  s = df.style.background_gradient(cmap=cm)
  s = s.set_precision(4)
  return s

In [14]:
# Please do not change this function
def dict_same_weight(w,ks):
  return dict(zip(ks,[w]*len(ks)))

In [15]:
# Please do not change this function
def split_question(Q, index, scales, softmax, filters):
  result = []
  for s in scales:
    q = QCACHE(Q(index=index, scale=s))
    for sf in softmax:
      for f in filters:
        if sf:
          qsf = QSOFTMAX(q,dim=[s])
        else:
          qsf = QPASS(q,descupdate={'softmax':''})
        qsf_f = QFILTER(qsf,filters[f],filtername=f)
        print(s,sf,f)
        result.append(qsf_f)
  return result

In [16]:
# Please do not change the model
mnli_pipelines = [
                  'typeform/distilbert-base-uncased-mnli'
]

In [17]:
# This is the model used for the evaluation. Do not change!:
mnli = pipeline("zero-shot-classification",device=device, model="typeform/distilbert-base-uncased-mnli")
mnli.model_identifier = "typeform/distilbert-base-uncased-mnli"

# **Intensifiers.**

Intensifiers are used to intensify the fraction or the frequency of a given sentence

For example:
*   *sentence:* Men eat more then women.
*   *fequency intensified:* Men **always/often/never** eat more then women.
*   *fraction intensified:* **all/most/few** Men eat more then women.

You can add new intensifiers, but please don't delete/change the existing ones.

In [18]:
#!export
intensifier_weights:SCALE = {
    'a bit':1,
    'slightly':1,
    'somewhat':1,
    'modereately':2,
    'rather':2,
    'pretty':3,
    'quite':3,
    'very':4,
    'extremely':4,
    'completely':5,
    'absolutely':5,
    'totally':5
}

frequency_weights:SCALE = {
    'never':1,
    'very rarely':1,
    'seldom':2,
    'rarely':2,
    'occasionally':3,
    'sometimes':3,
    'usually':3,
    'frequently':4,
    'often':4,
    'very frequently':5,
    'always':5,
    'constantly':5,
}

frequency_weights_after_emo:SCALE = {
    'during non of the days':1,
    'during a day or two':2,
    'for several days':2,
    'during some of the days':3,
    'several times':3,
    'less than half the time':3,
    'about half the time':3,
    'nearly half the time':3,
    'more than half the time':4,
    'often':4,
    'nearly every day':5,
    'every day':5,
    'very often':5,
    'all the time':5,
}

v2_frequency_weights:SCALE = {
    'during none of the days':1,
    'for several days':2,
    'more than half the time':3,
    'nearly every day':4,
}

frequency_weights_before_emo:SCALE = {
    'never':1,
    'very rarely':1,
    'during non of the days':1,
    'seldom':2,
    'rarely':2,
    'during a day or two':2,
    'for several days':2,
    'occasionally':3,
    'sometimes':3,
    'during some of the days':3,
    'several times':3,
    'less than half the time':3,
    'about half the time':3,
    'nearly half the time':3,
    'frequently':4,
    'more than half the time':4,
    'often':4,
    'very frequently':5,
    'always':5,
    'constantly':5,
    'nearly every day':5,
    'every day':5,
    'very often':5,
    'all the time':5,
}


intensifiers_fraction_without_none:SCALE={
            "few":1,
            "some":2,
            "many":3,
            "most":4,
            "all":5,
        }

intensifiers_fraction:SCALE={
            "none of the":1,
            "few":2,
            "some":3,
            "many":4,
            "most":4,
            "all":5,
        }



## You can add new intensifiers on your own
# Example:
# my_intensifier_dict = {
#     term1: <weight1>,
#     term2: <weight2>,
#     ...
# }


# Assignment details

## Background
Questionnaires are a popular research tool in social sciences as they are cost-effective and allow measurement of constructs that are otherwise difficult to assess. However, the response rate for questionnaires has decreased in recent years (Koen et al., 2018), and they have some limitations. For example, a questionnaire only provides a cross-sectional view of reality, making it difficult to capture the fluctuations in social reality (Einola & Alvesson, 2021). Additionally, respondents may be motivated to provide responses that misrepresent their true situation in order to achieve a desirable outcome, as they are aware that their responses can have an impact on their lives (Aday & Cornelius, 2006).

Language models are a statistical representation of the data they are trained on, and as such, they reflect the probability of word combinations in the training data. When language models are trained on texts created by humans, they can represent the mindset of the text creators (Garg et al. 2018).

Applying questionnaires on language models can be used for certification of these models.

## Your assignment
To translate questionnaires that are widely used in social sciences, so that they will be applied on language models. The translation will include phrases (or permutations) representing the possible responses to the questionnaire and assigned probability for each phrase.


## How to translate questions to MNLI queries?



1.   Read the item (question/statement) and possible responses.
2.   Identify and mark key phrases related to the construct being measured.
3.   Group together repeating key phrases (synonyms) into `dimensions` and give a name to each `dimension`. Possible dimesions, for example, may be gender, location, frequency, attitude, emotion, etc.
3.   Mark each dimesion as either `context`, `index`, or `scale`.
`context` terms set the scope for respondant, e.g., "at work" or "at home". Not all items have context terms.
`index` terms are highly related to the construct being measured by the item, e.g., "happy" or "sad". There should be at least one `index` term in each item.  
`scale` refers to intensifiers applied to the `index` terms, e.g., "very" happy or "somewhat" happy. Intensifiers may be present in the item or in the answers. If intensifiers are not present, they can always be added without modifying the measured construct.
4.   Reformulate the item using only one key phrase from each dimesion (discard repeated statemets in the same item).
5.   Reformulate item from step 4 in form of premise-hypotheis. The entailment of hypothesis from the premise is subjective and may be stronger for some people and weaker for others.
6.   Ensure that the template without key terms is bleached, i.e., expresses no attitude/stance.
7.   List additional key phrases in each dimesion that retain the original meaning of the item (synonyms).
8.   For `index` dimesions list additional terms with the opposite meaning (antonyms).
9.   Ensure diversity of intensifiers to cover the whole range on possible intensities e.g. from never to always.  
10.  Ensure that the template filled with any combination of terms from each dimesion are sound. You can ensure linguistic plausibility using respective models from HuggingFace.  
11.  Ensure that et least one set of key phrases is a paraphrase of the original item.  
11.  Define a dictionary for each dimension mapping a term to a number.
Map the values of `context` terms to 1.
Map the values of original `index` terms and their synonyms to 1 and the values of their antonyms to -1.  
Map the values of `intensifiers` to integers from 1 to 5.
12.  Run the question using `mnli` and check statistics printed by `report()`. Ensure that internal consistensy (silhouette) is above `0` -- the higher the better. Some MNLI models may result in low silhouette (`0.1`) while others will have high silhouette (`0.8`). We strive to have high silhouette on average.

#### Example of a question translation

**Original item:** Over the last 2 weeks, how often have you been bothered by the following problems?

Feeling nervous, anxious or on edge

**Answers:** Not at all, Several days, More than half the days, Nearly every day

Dimension 1 (index): bothered, problems, nervous, anxious, on edge

Dimension 2 (scale): Not at all, Several days, More than half the days, Nearly every day

The terms "bothered", "problem", "nervous", "anxious", and "on edge" all refer to something bad happening during the last two weeks. It is hard to formulate a consistent template where anxious can be replaced by bothered. Thus we will ignore terms bothered and problems and keep "nervous", "anxious", and "on edge".

Dimensions:
1.   Dimention 1 (index): `emotion={"nervous":1, "anxious":1, "on edge":1}`
2.   Dimension 2 (scale): `frequency={"Not at all":1, "Several days":2, "More than half the days":3, "Nearly every day":4}`

MNLI query example:
*   Premise: Over the last 2 weeks
*   Hypothesis: {More than half the days}  I was feeling {nervous}

Bleached MNLI query:
*   Premise: Over the last 2 weeks
*   Hypothesis: {frequency} I was feeling {emotion}


Lets code it and see the result

In [19]:
kw_emotion_bad = ['nervous','anxious','on an edge']
kw_emotion_good = ['calm','relaxed','in control']
dict_emotion = {
    'nervous':1,
    'anxious':1,
    'on an edge':1,
    'calm':-1,
    'relaxed':-1,
    'in control':-1,
}

dict_frequency = {
    "Not at all":1,
    "Several days":2,
    "More than half the days":3,
    "Nearly every day":4
}

class GAD2Q1(QMNLI):
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="Over the last 2 weeks",
        hypothesis_template="{frequency} I was feeling {emotion}",
        dimensions={"emotion":dict_emotion,
                    "frequency":dict_frequency,
        },
        descriptor = {"Questionnair":"GAD2",
                      "Factor":"",
                      "Ordinal":1,
                      "Original":'Over the last 2 weeks, how often have you been bothered by the following problems? Feeling nervous, anxious or on edge'
        },
        **kwargs,
    )

In [20]:
q = GAD2Q1()
q = q.run(mnli)
q.report(index=["emotion"],scale="frequency")

Query time: 2.3807930946350098
Mean score unfiltered: 4.162899119124631
Internal consistency (silhouette, correlation) for unfiltered: 0.5000479558161098
Internal consistency (Calinski&Harabasz)  for unfiltered: 4.418280861553245
Internal consistency (Davies&Bouldin) for unfiltered: 0.8303317627725408


index = ['emotion']
{'emotion', 'frequency'} {'frequency'} {'emotion'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


frequency,Not at all,Several days,More than half the days,Nearly every day
emotion,,,,
calm,0.0009,0.0005,0.3267,0.0460
in control,0.0010,0.0091,0.3873,0.2061
relaxed,0.0028,0.0013,0.2791,0.1176
anxious,0.0001,0.0314,0.6810,0.4297
nervous,0.0002,0.0581,0.4549,0.3399
on an edge,0.0006,0.0005,0.3445,0.2368


The result is not very good. We can see that all numbers are roughly equal and  silhouette is poor. Let's reformulate the hypothesis into something simpler without fancy intensifiers.  

In [21]:
kw_emotion_bad = ['nervous','anxious','on an edge']
kw_emotion_good = ['calm','relaxed','in control']
dict_emotion = {
    'nervous':1,
    'anxious':1,
    'on an edge':1,
    'calm':-1,
    'relaxed':-1,
    'in control':-1,
}

dict_frequency = frequency_weights

class GAD2Q1(QMNLI):
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="Over the last 2 weeks",
        hypothesis_template="I {frequency} felt {emotion}",
        dimensions={"emotion":dict_emotion,
                    "frequency":dict_frequency,
        },
        descriptor = {"Questionnair":"GAD2",
                      "Factor":"",
                      "Ordinal":1,
                      "Original":'Over the last 2 weeks, how often have you been bothered by the following problems? Feeling nervous, anxious or on edge'
        },
        **kwargs,
    )

In [22]:
q = GAD2Q1()
q = q.run(mnli)
q.report(index=["emotion"],scale="frequency")

Query time: 10.708878517150879
Mean score unfiltered: 8.957002379556798
Internal consistency (silhouette, correlation) for unfiltered: 0.5032085359752855
Internal consistency (Calinski&Harabasz)  for unfiltered: 4.498671829947358
Internal consistency (Davies&Bouldin) for unfiltered: 0.8537035957807471


index = ['emotion']
{'emotion', 'frequency'} {'frequency'} {'emotion'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


frequency,never,very rarely,rarely,seldom,occasionally,sometimes,usually,frequently,often,always,constantly,very frequently
emotion,,,,,,,,,,,,
calm,0.0066,0.2612,0.4172,0.2080,0.6921,0.5916,0.3671,0.2244,0.3394,0.0413,0.0009,0.1231
in control,0.0001,0.0755,0.1976,0.0435,0.6390,0.6399,0.4295,0.4145,0.4746,0.2727,0.2651,0.3641
relaxed,0.0015,0.2362,0.3557,0.2546,0.6779,0.5825,0.3509,0.3273,0.3759,0.1517,0.0345,0.2506
anxious,0.0003,0.0503,0.1116,0.0056,0.6903,0.6932,0.3835,0.5344,0.5402,0.3173,0.3252,0.5214
nervous,0.0120,0.1148,0.2005,0.0700,0.5963,0.5759,0.3651,0.4788,0.4752,0.3147,0.3157,0.4635
on an edge,0.0001,0.1460,0.1818,0.0102,0.6408,0.5506,0.3603,0.3921,0.4288,0.2751,0.3057,0.3809


These results are better (`silhouette=0.3`).

Lets see what happens if we apply `softmax` on the intensifiers.

In [23]:
q = QSOFTMAX(q,dim=['frequency'])
q = q.run(mnli)
q.report(index=["emotion"],scale="frequency")

QSOFTMAX delgates execution of run(..) to GAD2Q1
Query time: 6.669272422790527
Mean score unfiltered: 0.4047456346452236
Internal consistency (silhouette, correlation) for unfiltered: 0.5029600477192934
Internal consistency (Calinski&Harabasz)  for unfiltered: 4.619473668478878
Internal consistency (Davies&Bouldin) for unfiltered: 0.8460866702477179


index = ['emotion']
{'emotion', 'frequency'} {'frequency'} {'emotion'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


frequency,never,very rarely,rarely,seldom,occasionally,sometimes,usually,frequently,often,always,constantly,very frequently
emotion,,,,,,,,,,,,
calm,0.0624,0.0805,0.0941,0.0763,0.1238,0.1120,0.0895,0.0776,0.0870,0.0646,0.0620,0.0701
in control,0.0594,0.0640,0.0723,0.0620,0.1125,0.1126,0.0912,0.0899,0.0954,0.0780,0.0774,0.0854
relaxed,0.0607,0.0768,0.0865,0.0782,0.1194,0.1086,0.0861,0.0841,0.0883,0.0706,0.0628,0.0779
anxious,0.0572,0.0601,0.0639,0.0575,0.1139,0.1143,0.0838,0.0975,0.0981,0.0785,0.0791,0.0962
nervous,0.0595,0.0659,0.0718,0.0630,0.1067,0.1045,0.0847,0.0949,0.0945,0.0805,0.0806,0.0934
on an edge,0.0603,0.0698,0.0723,0.0609,0.1144,0.1046,0.0864,0.0892,0.0926,0.0794,0.0819,0.0882


Now lets try a more complex question:

Original question:   B(P) 20. {Men} {should be} willing to sacrifice their own well being in order to provide financially for the {women} in their lives.


In [24]:
kw_attitude_pos = ['sacrifice', 'give up', "spare", "offer up"]
kw_attitude_neg = ['preserve', 'retain', "withhold", "magnify"]
dict_attitude = dict(dict_same_weight(1,kw_attitude_pos), **dict_same_weight(-1,kw_attitude_neg))

kw_with = ["should", ]
kw_without = ["should not", ]
dict_relationship = dict(dict_same_weight(1,kw_with), **dict_same_weight(-1,kw_without))

kw_male = ["men"]
kw_female = ["women"]
dict_gender = dict_same_weight(1,kw_male+kw_female)



class ASIQ20(QMNLI):
  """
  B(P) 20. {Men} {should be} willing to sacrifice their own well being in order to provide financially for the {women} in their lives.

  MNLI Query example:
        context="in order to provide financially for the {women} in their lives",
        template="{men} {should} {usually} {sacrifice} their own well being",
  """
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="in order to provide financially for the {gender2} in their lives",
        hypothesis_template="{frequency} {gender1} {relationship} {attitude} their own well being",
        dimensions={
            "gender1":dict_gender,
            "gender2":dict_gender,
            "frequency":frequency_weights,
            "attitude":dict_attitude,
            "relationship":dict_relationship,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BP",
                      "Ordinal":20,
                      "Original":'B(P) 20. {Men} {should be} willing to sacrifice their own well being in order to provide financially for the {women} in their lives.'
        },
        **kwargs,
    )

In [25]:
q = ASIQ20()
q = QSOFTMAX(q,dim=['frequency'])
q = q.run(mnli)
q.report(index=["attitude"],scale="frequency",filters={"men should sacrifice for women": {"gender1":kw_male,"gender2":kw_female}})
# linguistic_df = linguistic_acceptabilities(q, index=["attitude"], scale="frequency",question_name='GAD2Q1', student_id=student_id)
# linguistic_df[['cola_score','semantic_similarity','silhouette_score']].mean(axis=0)

QSOFTMAX delgates execution of run(..) to ASIQ20
Query time: 58.251009702682495
Mean score men should sacrifice for women: 0.013914208859205246
Internal consistency (silhouette, correlation) for men should sacrifice for women: -0.09275112464344651
Internal consistency (Calinski&Harabasz)  for men should sacrifice for women: 0.36705556736284833
Internal consistency (Davies&Bouldin) for men should sacrifice for women: 3.7974966391201894


index = ['attitude']
{'relationship', 'frequency', 'attitude', 'gender2', 'gender1'} {'frequency'} {'attitude'}
{'relationship', 'gender1', 'gender2'}


<ipython-input-8-a6859e5ab206>:351: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  W = W[aggregated_fields]
<ipython-input-8-a6859e5ab206>:353: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["P"] = df["P"]*W
<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


frequency,never,very rarely,rarely,seldom,occasionally,sometimes,usually,frequently,often,always,constantly,very frequently
attitude,,,,,,,,,,,,
magnify,0.0019,0.0016,0.0087,0.0088,-0.0048,-0.0009,-0.0036,-0.0007,-0.0084,0.0016,0.0016,-0.0059
preserve,-0.0024,-0.0045,0.0024,0.0101,-0.0028,0.0068,0.0008,0.0021,-0.0021,-0.0025,-0.0026,-0.0051
retain,-0.0041,-0.0069,-0.0030,0.0032,0.0007,0.0110,0.0034,0.0041,0.0029,-0.0035,-0.0039,-0.0038
withhold,0.0006,0.0029,0.0071,0.0148,-0.0033,-0.0136,-0.0031,-0.0049,-0.0066,0.0032,0.0097,-0.0067
give up,-0.0043,0.0004,0.0063,0.0075,0.0030,-0.0017,-0.0037,-0.0060,-0.0057,0.0027,0.0070,-0.0054
offer up,0.0021,-0.0024,0.0086,0.0145,-0.0011,-0.0009,-0.0002,-0.0077,-0.0048,0.0011,0.0012,-0.0102
sacrifice,-0.0016,-0.0014,0.0033,0.0078,0.0023,-0.0021,-0.0039,-0.0045,-0.0047,0.0030,0.0039,-0.0021
spare,0.0009,-0.0044,0.0028,0.0075,-0.0019,-0.0022,0.0005,-0.0001,-0.0015,0.0012,0.0012,-0.0041


Download the list of all questions and questionnairs and store them in a local Pandas DataFrame.

These are the questions you should translate into MNLI quesrries

## Assignment solution

Instractions for how to submmit questions.

*   Replace **\<QuestionnairName>** **\<QuestionNumber>** for each question using **Questionnaire** and **Question** Number
*   Do not forget to update the descriptor
*   Update the \<question_id>
*   **Importent** your question will be tested using the index and scale you provided for them in the all_my_question dictioneries



In [26]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

if IN_COLAB:
  url = "https://drive.google.com/uc?id=1oKnlIUXWK1_WCd2XZ6oBJXquqRJkNQQc&export=download"

  all_questionnairs = pd.read_csv(url)
  all_questionnairs = all_questionnairs.set_index("QuestionID")

  questions_to_be_translated = all_questionnairs.sample(n=n_questions, random_state=student_id)

In [27]:
questions_to_be_translated


,Questionnaire,Factor,Question Number,Context,Context\Term,Intensifire
QuestionID,,,,,,
299,MBI,Emotional Exhaustion,3.,During the last year,I feel fatigued when I get up in the morning and have to face another day on the job.,0 Never‚ 1 A few times a year‚ 2 Once a month‚ 3 A few times per month‚ 4 Once a week‚ 5 A few times per week‚ 6 Every day
205,WHOQOL,Social Relations- Personal relationships,15,During the last two weeks,How satisfied are you with your capacity for work?,(1) Very dissatisfied; (2) Dissatisfied; (3) Neither satisfied nor dissatisfied; (4) Satisfied; (5) very satisfied
350,CD-RISC,persistence,6,NaN,I can achieve goals despite obstacles,0 (not true at all) to 4 (true nearly all the time)
135,COPE inventory,Use of emotional social support,52,when YOU experience a stressful event,I talk to someone about how I feel.,1 = I usually don't do this at all\n2 = I usually do this a little bit\n3 = I usually do this a medium amount\n4 = I usually do this a lot
274,Portrait Values Questionnaire (PVQ),Universalism,19,NaN,He strongly believes that people should care for nature.,Not at All Like Me‚ Not Like Me‚ A Little Like Me‚ Somewhat Like Me‚ Like Me‚ Very Much Like Me
28,Modern Racism scale,NaN,6,NaN,"Over the past few years, blacks have gotten more economically than they deserve.*",agree strongly; 1 – disagree somewhat; 2 – disagree slightly; 3 – agree slightly; 4 – agree somewhat; 5 – agree strongly
281,Portrait Values Questionnaire (PVQ),Hedonism,26,NaN,He thinks it is best to do things in traditional ways. It is important to him to keep up the customs he has learned.,Not at All Like Me‚ Not Like Me‚ A Little Like Me‚ Somewhat Like Me‚ Like Me‚ Very Much Like Me
237,SOC,Comprehensibility,21.,NaN,Does it happen that you have feelings inside you would rather not feel?,very often 1 2 3 4 5 6 7 very seldom or never
338,CCRAM,Place Attachment,4,In my community,I am proud to tell others where I live.,strongly disagree- 1234567- strongly agree


In [28]:
all_my_questions = {}

## Q1

In [68]:
#!export
#QuestionID from the questions_to_be_translated data frame
question_id = 299

#index
kw_index_original = ["tiredness", "exhaustion", "weariness", "fatigueness", "weakness", "tired-out", "drained"] # add keywords for original question
kw_index_oposite = ["energy", "vigor", "vitality", "robustness", "zeal", "refreshment", "rejuvenation"] # add keywords for oposite question
dict_index = dict(dict_same_weight(1,kw_index_original), **dict_same_weight(-1,kw_index_oposite))

intensifiers_fraction1:SCALE={
            "none":1,
            "few":2,
            "some":3,
            "many":4,
            "most":5,
        }

#scale
dict_scale = intensifiers_fraction1 # change if you want a diffrent scale

#optional context dimensions
kw_some_context = []
dict_some_context = dict_same_weight(1,kw_some_context)


class MBIQ3(QMNLI):
  """
  Write here the original question

  MNLI Query example:
        premise= write here an example of a premise ,
        hypothesis= write here an example of a hypothesis,
  """
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="{scale} of the days last year,",
        hypothesis_template="i felt {index} getting up in the morning and have to face another day on the job .",
        dimensions={
            "index":dict_index,
            "scale":dict_scale,
        },
        descriptor = {"Questionnair":"MBI",
                      "Factor":"Emotional Exhaustion",
                      "Ordinal":3,
                      "Original":'During the last year, I feel fatigued when I get up in the morning and have to face another day on the job.',
        },
        **kwargs,
    )

#instantiation
q = MBIQ3()
#running the question with some model stored in mnli
q = q.run(mnli)
#printing the model answers. You may use filters to report the question results in different contexts.
index = ["index"]
scale = "scale"
q.report(index=index,scale=scale,filters={"unfiltered":{}})
#Add the question with its index and scale to the list of all questions.
all_my_questions[(question_id, student_id)] = (q, index, scale)
print_permutations(q)
check_question_not_default(q, q.get_class_name())

Query time: 12.513561010360718
Mean score unfiltered: 9.954121238013613
Internal consistency (silhouette, correlation) for unfiltered: 0.5026895696802405
Internal consistency (Calinski&Harabasz)  for unfiltered: 12.971911577519666
Internal consistency (Davies&Bouldin) for unfiltered: 0.7842693011312205


index = ['index']
{'scale', 'index'} {'scale'} {'index'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


scale,none,few,some,many,most
index,,,,,
energy,0.0272,0.0706,0.0697,0.0674,0.0648
refreshment,0.0666,0.1478,0.1479,0.1258,0.1158
rejuvenation,0.0121,0.0321,0.0424,0.0447,0.0386
robustness,0.0003,0.0041,0.0040,0.0038,0.0027
vigor,0.0193,0.0528,0.0583,0.0509,0.0518
vitality,0.0002,0.0096,0.0106,0.0103,0.0060
zeal,0.0322,0.0920,0.0758,0.0652,0.0635
drained,0.0696,0.1171,0.1365,0.1238,0.1207
exhaustion,0.0311,0.0407,0.0652,0.0584,0.0596


{'Questionnair': 'MBI', 'Factor': 'Emotional Exhaustion', 'Ordinal': 3, 'Original': 'During the last year, I feel fatigued when I get up in the morning and have to face another day on the job.', 'scale': 'intensifier', 'index': "['scale', 'index']", 'query': '{scale} of the days last year,->i felt {index} getting up in the morning and have to face another day on the job .'}
0. none of the days last year, -> i felt tiredness getting up in the morning and have to face another day on the job . 1
1. few of the days last year, -> i felt tiredness getting up in the morning and have to face another day on the job . 2
2. some of the days last year, -> i felt tiredness getting up in the morning and have to face another day on the job . 3
3. many of the days last year, -> i felt tiredness getting up in the morning and have to face another day on the job . 4
4. most of the days last year, -> i felt tiredness getting up in the morning and have to face another day on the job . 5
5. none of the days

True

## Q2

In [80]:
#!export
#QuestionID from the questions_to_be_translated data frame
question_id = 205

#index
kw_index_original = ["Pleased", "Satisfied", "Gratified", "Happy", "Delighted"]
kw_index_oposite = ["Unsatisfied", "Displeased", "Unhappy", "Frustrated"] # add keywords for oposite question
# add keywords for original question
dict_index = dict(dict_same_weight(1,kw_index_original), **dict_same_weight(-1,kw_index_oposite))
#scale
intensifier_weights1:SCALE = {
    'somewhat':1,
    'pretty':2,
    'quite':3,
    'very':4,
    'extremely':4,
    'absolutely':5,
}
dict_scale = intensifier_weights1 # change if you want a diffrent scale

#optional context dimensions
kw_some_context = []
dict_some_context = dict_same_weight(1,kw_some_context)


class WHOQOLQ15(QMNLI):
  """
  Write here the original question

  MNLI Query example:
        premise= write here an example of a premise ,
        hypothesis= write here an example of a hypothesis,
  """
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="Over the last two weeks,",
        hypothesis_template="I feel {scale} {index} with my capacity of work",
        dimensions={
            "index":dict_index,
            "scale":dict_scale,
        },
        descriptor = {"Questionnair":"WHOQOL",
                      "Factor":"Social Relations- Personal relationships",
                      "Ordinal":15,
                      "Original":'During the last two weeks, How satisfied are you with your capacity for work?',
        },
        **kwargs,
    )

#instantiation
q = WHOQOLQ15()
#running the question with some model stored in mnli
q = q.run(mnli)
#printing the model answers. You may use filters to report the question results in different contexts.
index = ["index"]
scale = "scale"
q.report(index=index,scale=scale,filters={"unfiltered":{}})
#Add the question with its index and scale to the list of all questions.
all_my_questions[(question_id, student_id)] = (q, index, scale)
print_permutations(q)
check_question_not_default(q, q.get_class_name())

Query time: 8.906550884246826
Mean score unfiltered: -9.674759849905968
Internal consistency (silhouette, correlation) for unfiltered: 0.5456434537296851
Internal consistency (Calinski&Harabasz)  for unfiltered: 6.489145473809742
Internal consistency (Davies&Bouldin) for unfiltered: 0.8833116386484902


index = ['index']
{'scale', 'index'} {'scale'} {'index'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


scale,somewhat,pretty,quite,extremely,very,absolutely
index,,,,,,
Displeased,0.8585,0.7106,0.6996,0.7399,0.7507,0.7280
Frustrated,0.5893,0.3518,0.3073,0.3546,0.3930,0.3224
Unhappy,0.8155,0.6330,0.5842,0.6261,0.6571,0.6092
Unsatisfied,0.8164,0.6032,0.5645,0.6467,0.5841,0.5598
Delighted,0.6505,0.4348,0.4750,0.4676,0.4914,0.5053
Gratified,0.8663,0.3639,0.4226,0.3163,0.3239,0.2673
Happy,0.7064,0.3117,0.3482,0.2877,0.2992,0.2729
Pleased,0.6679,0.3492,0.3773,0.3492,0.4044,0.3656
Satisfied,0.6990,0.2479,0.3198,0.2148,0.2797,0.1896


{'Questionnair': 'WHOQOL', 'Factor': 'Social Relations- Personal relationships', 'Ordinal': 15, 'Original': 'During the last two weeks, How satisfied are you with your capacity for work?', 'scale': 'intensifier', 'index': "['scale', 'index']", 'query': 'Over the last two weeks,->I feel {scale} {index} with my capacity of work'}
0. Over the last two weeks, -> I feel somewhat Pleased with my capacity of work 1
1. Over the last two weeks, -> I feel pretty Pleased with my capacity of work 2
2. Over the last two weeks, -> I feel quite Pleased with my capacity of work 3
3. Over the last two weeks, -> I feel very Pleased with my capacity of work 4
4. Over the last two weeks, -> I feel extremely Pleased with my capacity of work 4
5. Over the last two weeks, -> I feel absolutely Pleased with my capacity of work 5
6. Over the last two weeks, -> I feel somewhat Satisfied with my capacity of work 1
7. Over the last two weeks, -> I feel pretty Satisfied with my capacity of work 2
8. Over the last t

True

## Q3

In [31]:
#!export
#QuestionID from the questions_to_be_translated data frame
question_id = 350

#index
kw_index_original = ["Managed", "Handled", "Controlled", "Directed", "Supervised", "Organized", "Administered", "Conducted", "Governed", "Led", "Executed"] # add keywords for original question
kw_index_oposite = ["Mismanaged", "Mishandled", "Neglected", "Abandoned", "Failed", "Botched", "Misdirected", "Overlooked", "Bungled", "Fumbled"] # add keywords for oposite question
dict_index = dict(dict_same_weight(1,kw_index_original), **dict_same_weight(-1,kw_index_oposite))

#scale
dict_scale = frequency_weights # change if you want a diffrent scale

#optional context dimensions
kw_some_context = []
dict_some_context = dict_same_weight(1,kw_some_context)


class CDQ6(QMNLI):
  """
  Write here the original question

  MNLI Query example:
        premise= write here an example of a premise ,
        hypothesis= write here an example of a hypothesis,
  """
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="Over the last few years,",
        hypothesis_template="I {scale} {index} to achieve goals despite obstacles",
        dimensions={
            "index":dict_index,
            "scale":dict_scale,
        },
        descriptor = {"Questionnair":"CD-RISC",
                      "Factor":"persistence",
                      "Ordinal":6,
                      "Original":'I can achieve goals despite obstacles',
        },
        **kwargs,
    )

#instantiation
q = CDQ6()
#running the question with some model stored in mnli
q = q.run(mnli)
#printing the model answers. You may use filters to report the question results in different contexts.
index = ["index"]
scale = "scale"
q.report(index=index,scale=scale,filters={"unfiltered":{}})
#Add the question with its index and scale to the list of all questions.
all_my_questions[(question_id, student_id)] = (q, index, scale)
print_permutations(q)
check_question_not_default(q, q.get_class_name())

Query time: 27.828940868377686
Mean score unfiltered: 19.60730443201828
Internal consistency (silhouette, correlation) for unfiltered: 0.6417538208675211
Internal consistency (Calinski&Harabasz)  for unfiltered: 13.876410653430609
Internal consistency (Davies&Bouldin) for unfiltered: 1.071766926019115


index = ['index']
{'scale', 'index'} {'scale'} {'index'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


scale,never,very rarely,rarely,seldom,occasionally,sometimes,usually,frequently,often,always,constantly,very frequently
index,,,,,,,,,,,,
Abandoned,0.3796,0.4628,0.5294,0.5387,0.5078,0.4586,0.0787,0.0457,0.1320,0.0000,0.0001,0.0570
Botched,0.1008,0.3963,0.4399,0.3457,0.6874,0.6141,0.3308,0.2933,0.3493,0.0462,0.0136,0.2747
Bungled,0.0001,0.1580,0.2065,0.1382,0.4024,0.4009,0.2450,0.1959,0.2399,0.0863,0.0696,0.1618
Neglected,0.3234,0.4425,0.4528,0.4458,0.5939,0.5335,0.2537,0.1885,0.2803,0.0006,0.0002,0.1407
Mismanaged,0.2566,0.4067,0.4529,0.3820,0.7114,0.6961,0.3658,0.3738,0.4367,0.1293,0.0763,0.3820
Mishandled,0.3317,0.4272,0.4764,0.4268,0.6773,0.6894,0.4261,0.3986,0.4624,0.2106,0.1037,0.4133
Misdirected,0.3887,0.4504,0.5157,0.4882,0.6836,0.7376,0.4091,0.3458,0.4419,0.1818,0.0802,0.2757
Failed,0.2585,0.4238,0.4862,0.4715,0.6070,0.6253,0.3186,0.3802,0.4155,0.0087,0.0009,0.3520
Fumbled,0.4631,0.3998,0.4163,0.4923,0.3429,0.3278,0.1545,0.1452,0.1823,0.0474,0.0387,0.1501


{'Questionnair': 'CD-RISC', 'Factor': 'persistence', 'Ordinal': 6, 'Original': 'I can achieve goals despite obstacles', 'scale': 'intensifier', 'index': "['scale', 'index']", 'query': 'Over the last few years,->I {scale} {index} to achieve goals despite obstacles'}
0. Over the last few years, -> I never Managed to achieve goals despite obstacles 1
1. Over the last few years, -> I very rarely Managed to achieve goals despite obstacles 1
2. Over the last few years, -> I seldom Managed to achieve goals despite obstacles 2
3. Over the last few years, -> I rarely Managed to achieve goals despite obstacles 2
4. Over the last few years, -> I occasionally Managed to achieve goals despite obstacles 3
5. Over the last few years, -> I sometimes Managed to achieve goals despite obstacles 3
6. Over the last few years, -> I usually Managed to achieve goals despite obstacles 3
7. Over the last few years, -> I frequently Managed to achieve goals despite obstacles 4
8. Over the last few years, -> I oft

True

## Q4

In [32]:
#!export
#QuestionID from the questions_to_be_translated data frame
question_id = 135

#index
kw_index_original = ["Discuss", "Share", "Express", "Converse", "Confide", "Open up", "Speak", "Chat"] # add keywords for original question
kw_index_oposite = ["Conceal", "Refrain", "Avoid", "Mute", "Disguise"] # add keywords for oposite question
dict_index = dict(dict_same_weight(1,kw_index_original), **dict_same_weight(-1,kw_index_oposite))

#scale
freq4:SCALE = {
    'never':1,
    'very rarely':1,
    'seldom':2,
    'rarely':2,
    'frequently':4,
    'often':4,
    'very frequently':5,
    'always':5,
    'constantly':5,
}

dict_scale = freq4
 # change if you want a diffrent scale

#optional context dimensions
kw_some_context = []
dict_some_context = dict_same_weight(1,kw_some_context)


class CIQ52(QMNLI):
  """
  Write here the original question

  MNLI Query example:
        premise= write here an example of a premise ,
        hypothesis= write here an example of a hypothesis,
  """
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="when i experience a stressful event,",
        hypothesis_template="I'm {scale} willing to {index} my feelings",
        dimensions={
            "index":dict_index,
            "scale":dict_scale,
        },
        descriptor = {"Questionnair":"COPE inventory",
                      "Factor":"Use of emotional social support",
                      "Ordinal":52,
                      "Original":'when you experience a stressful event, I talk to someone about how I feel',
        },
        **kwargs,
    )

#instantiation
q = CIQ52()
#running the question with some model stored in mnli
q = q.run(mnli)
#printing the model answers. You may use filters to report the question results in different contexts.
index = ["index"]
scale = "scale"
q.report(index=index,scale=scale,filters={"unfiltered":{}})
#Add the question with its index and scale to the list of all questions.
all_my_questions[(question_id, student_id)] = (q, index, scale)
print_permutations(q)
check_question_not_default(q, q.get_class_name())

Query time: 21.709540128707886
Mean score unfiltered: 5.9795434944098815
Internal consistency (silhouette, correlation) for unfiltered: 0.8219900173893324
Internal consistency (Calinski&Harabasz)  for unfiltered: 30.485599439045764
Internal consistency (Davies&Bouldin) for unfiltered: 0.56705886163665


index = ['index']
{'scale', 'index'} {'scale'} {'index'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


scale,never,very rarely,rarely,seldom,frequently,often,always,constantly,very frequently
index,,,,,,,,,
Avoid,0.1362,0.1947,0.3420,0.2968,0.2646,0.2955,0.0479,0.0909,0.2284
Conceal,0.1284,0.1343,0.2770,0.2740,0.3471,0.4084,0.0610,0.1287,0.1788
Disguise,0.1480,0.2417,0.3777,0.3922,0.3724,0.4199,0.1615,0.1926,0.2140
Mute,0.0829,0.1865,0.3864,0.3554,0.2486,0.2773,0.0061,0.0562,0.1565
Refrain,0.1923,0.1267,0.2787,0.2692,0.1720,0.2767,0.0081,0.0056,0.1665
Chat,0.0003,0.0014,0.0270,0.0094,0.2407,0.3071,0.0205,0.0446,0.1180
Confide,0.0020,0.0066,0.0771,0.0433,0.3383,0.3976,0.0926,0.1304,0.1785
Converse,0.0120,0.0257,0.1133,0.0678,0.2753,0.3464,0.0462,0.0757,0.1562
Discuss,0.0014,0.0036,0.0221,0.0057,0.3704,0.4229,0.0613,0.1305,0.1567


{'Questionnair': 'COPE inventory', 'Factor': 'Use of emotional social support', 'Ordinal': 52, 'Original': 'when you experience a stressful event, I talk to someone about how I feel', 'scale': 'intensifier', 'index': "['scale', 'index']", 'query': "when i experience a stressful event,->I'm {scale} willing to {index} my feelings"}
0. when i experience a stressful event, -> I'm never willing to Discuss my feelings 1
1. when i experience a stressful event, -> I'm very rarely willing to Discuss my feelings 1
2. when i experience a stressful event, -> I'm seldom willing to Discuss my feelings 2
3. when i experience a stressful event, -> I'm rarely willing to Discuss my feelings 2
4. when i experience a stressful event, -> I'm frequently willing to Discuss my feelings 4
5. when i experience a stressful event, -> I'm often willing to Discuss my feelings 4
6. when i experience a stressful event, -> I'm very frequently willing to Discuss my feelings 5
7. when i experience a stressful event, -> 

True

## Q5

In [33]:
#!export
#QuestionID from the questions_to_be_translated data frame
question_id = 274

#index
kw_index_original = ["Ought to have", "Should've", "Supposed to have", "Expected to have", "Required to have", "Meant to have", "Needed to have", "Obliged to have", "Demanded to have", "Recommended to have"] # add keywords for original question
kw_index_oposite = ["Should not have", "Ought not to have", "Supposed not to have", "Not expected to have", "Not required to have", "Not meant to have", "Not needed to have", "Not obliged to have", "Not demanded to have", "Not recommended to have"] # add keywords for oposite question
dict_index = dict(dict_same_weight(1,kw_index_original), **dict_same_weight(-1,kw_index_oposite))

#scale
dict_scale = intensifier_weights # change if you want a diffrent scale

#optional context dimensions
kw_some_context = []
dict_some_context = dict_same_weight(1,kw_some_context)


class PVQ19(QMNLI):
  """
  Write here the original question

  MNLI Query example:
        premise= write here an example of a premise ,
        hypothesis= write here an example of a hypothesis,
  """
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="in my opinion,",
        hypothesis_template="people {index} {scale} care for nature",
        dimensions={
            "index":dict_index,
            "scale":dict_scale,
        },
        descriptor = {"Questionnair":"Portarit Values Questionnarie (PVQ)",
                      "Factor":"Universalism",
                      "Ordinal":19,
                      "Original":'He strongly believes that people should care for nature',
        },
        **kwargs,
    )

#instantiation
q = PVQ19()
#running the question with some model stored in mnli
q = q.run(mnli)
#printing the model answers. You may use filters to report the question results in different contexts.
index = ["index"]
scale = "scale"
q.report(index=index,scale=scale,filters={"unfiltered":{}})
#Add the question with its index and scale to the list of all questions.
all_my_questions[(question_id, student_id)] = (q, index, scale)
print_permutations(q)
check_question_not_default(q, q.get_class_name())

Query time: 18.38634943962097
Mean score unfiltered: 92.5624232927803
Internal consistency (silhouette, correlation) for unfiltered: 0.8302400664300185
Internal consistency (Calinski&Harabasz)  for unfiltered: 37.219152269622306
Internal consistency (Davies&Bouldin) for unfiltered: 0.597068104303899


index = ['index']
{'scale', 'index'} {'scale'} {'index'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


scale,a bit,slightly,somewhat,modereately,rather,pretty,quite,extremely,very,absolutely,completely,totally
index,,,,,,,,,,,,
Not recommended to have,0.0012,0.0232,0.0078,0.0009,0.0017,0.0288,0.0490,0.0005,0.0022,0.0190,0.3475,0.2439
Should not have,0.0053,0.0316,0.0295,0.1015,0.0287,0.2491,0.1298,0.0182,0.1618,0.1349,0.4490,0.3601
Not demanded to have,0.0013,0.0036,0.0073,0.0020,0.0039,0.0064,0.0148,0.0008,0.0016,0.0034,0.0331,0.0156
Not expected to have,0.3826,0.4363,0.4896,0.3839,0.3851,0.3300,0.3765,0.2855,0.2940,0.3732,0.4307,0.4123
Not meant to have,0.2596,0.3242,0.3495,0.3356,0.2985,0.3136,0.3047,0.2393,0.2811,0.3190,0.4566,0.4161
Not needed to have,0.0022,0.0316,0.0113,0.0486,0.0214,0.0404,0.0266,0.0022,0.0046,0.0139,0.1068,0.0931
Not obliged to have,0.0029,0.0024,0.0029,0.0040,0.0084,0.0167,0.0148,0.0011,0.0039,0.0356,0.5081,0.4245
Supposed not to have,0.1213,0.1981,0.2605,0.1538,0.1454,0.2043,0.1790,0.1109,0.1810,0.2035,0.3324,0.2836
Not required to have,0.0152,0.0885,0.0618,0.2585,0.1076,0.0600,0.0306,0.0069,0.0063,0.0096,0.1259,0.1287


{'Questionnair': 'Portarit Values Questionnarie (PVQ)', 'Factor': 'Universalism', 'Ordinal': 19, 'Original': 'He strongly believes that people should care for nature', 'scale': 'intensifier', 'index': "['scale', 'index']", 'query': 'in my opinion,->people {index} {scale} care for nature'}
0. in my opinion, -> people Ought to have a bit care for nature 1
1. in my opinion, -> people Ought to have slightly care for nature 1
2. in my opinion, -> people Ought to have somewhat care for nature 1
3. in my opinion, -> people Ought to have modereately care for nature 2
4. in my opinion, -> people Ought to have rather care for nature 2
5. in my opinion, -> people Ought to have pretty care for nature 3
6. in my opinion, -> people Ought to have quite care for nature 3
7. in my opinion, -> people Ought to have very care for nature 4
8. in my opinion, -> people Ought to have extremely care for nature 4
9. in my opinion, -> people Ought to have completely care for nature 5
10. in my opinion, -> people

True

## Q6

In [43]:
#!export
#QuestionID from the questions_to_be_translated data frame
question_id = 28

#index
kw_index_original = ["Ought to have", "Should've", "Supposed to have", "Expected to have", "Required to have", "Meant to have", "Needed to have", "Obliged to have", "Demanded to have", "Recommended to have"] # add keywords for original question
kw_index_oposite = ["Should not have", "Ought not to have", "Supposed not to have", "Not expected to have", "Not required to have", "Not meant to have", "Not needed to have", "Not obliged to have", "Not demanded to have", "Not recommended to have"] # add keywords for oposite question
dict_index = dict(dict_same_weight(1,kw_index_original), **dict_same_weight(-1,kw_index_oposite))

#scale
dict_scale = intensifier_weights # change if you want a diffrent scale

#optional context dimensions
kw_some_context = []
dict_some_context = dict_same_weight(1,kw_some_context)


class MRSQ6(QMNLI):
  """
  Write here the original question

  MNLI Query example:
        premise= write here an example of a premise ,
        hypothesis= write here an example of a hypothesis,
  """
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="Over the past few years,",
        hypothesis_template="blacks have gotten {scale} economically than they {index}",
        dimensions={
            "index":dict_index,
            "scale":dict_scale,
        },
        descriptor = {"Questionnair":"Modern Racsim scale",
                      "Factor":"",
                      "Ordinal":6,
                      "Original":'Over the past few years, blacks have gotten more economically than they deserve.',
        },
        **kwargs,
    )

#instantiation
q = MRSQ6()
#running the question with some model stored in mnli
q = q.run(mnli)
#printing the model answers. You may use filters to report the question results in different contexts.
index = ["index"]
scale = "scale"
q.report(index=index,scale=scale,filters={"unfiltered":{}})
#Add the question with its index and scale to the list of all questions.
all_my_questions[(question_id, student_id)] = (q, index, scale)
print_permutations(q)
check_question_not_default(q, q.get_class_name())

Query time: 19.628849267959595
Mean score unfiltered: -41.64953863620758
Internal consistency (silhouette, correlation) for unfiltered: 0.638341590360999
Internal consistency (Calinski&Harabasz)  for unfiltered: 23.008848801187625
Internal consistency (Davies&Bouldin) for unfiltered: 0.7801661431089755


index = ['index']
{'scale', 'index'} {'scale'} {'index'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


scale,a bit,slightly,somewhat,modereately,rather,pretty,quite,extremely,very,absolutely,completely,totally
index,,,,,,,,,,,,
Not recommended to have,0.7247,0.5981,0.7689,0.6190,0.4467,0.6118,0.7898,0.7458,0.7411,0.6545,0.5767,0.5422
Should not have,0.7910,0.6556,0.8624,0.7014,0.4891,0.6920,0.8348,0.7994,0.7800,0.7248,0.6493,0.6060
Not demanded to have,0.6410,0.5762,0.7565,0.5872,0.4698,0.5563,0.7308,0.7032,0.6842,0.6110,0.5402,0.5095
Not expected to have,0.7404,0.6180,0.7935,0.6000,0.4904,0.6422,0.8056,0.7638,0.7599,0.6839,0.5955,0.5734
Not meant to have,0.7479,0.6158,0.8106,0.6056,0.4878,0.6165,0.7676,0.7211,0.7205,0.6132,0.5519,0.5132
Not needed to have,0.6318,0.5458,0.7412,0.5413,0.4234,0.5753,0.7456,0.6868,0.6792,0.6237,0.5275,0.4894
Not obliged to have,0.7437,0.5620,0.7948,0.6331,0.4313,0.6913,0.8303,0.7561,0.7602,0.7025,0.6268,0.5737
Supposed not to have,0.7411,0.5797,0.7723,0.5612,0.4571,0.6152,0.7958,0.7587,0.7468,0.6884,0.6095,0.5779
Not required to have,0.7195,0.5812,0.7816,0.6292,0.5327,0.5933,0.7844,0.7307,0.7243,0.6430,0.5414,0.5096


{'Questionnair': 'Modern Racsim scale', 'Factor': '', 'Ordinal': 6, 'Original': 'Over the past few years, blacks have gotten more economically than they deserve.', 'scale': 'intensifier', 'index': "['scale', 'index']", 'query': 'Over the past few years,->blacks have gotten {scale} economically than they {index}'}
0. Over the past few years, -> blacks have gotten a bit economically than they Ought to have 1
1. Over the past few years, -> blacks have gotten slightly economically than they Ought to have 1
2. Over the past few years, -> blacks have gotten somewhat economically than they Ought to have 1
3. Over the past few years, -> blacks have gotten modereately economically than they Ought to have 2
4. Over the past few years, -> blacks have gotten rather economically than they Ought to have 2
5. Over the past few years, -> blacks have gotten pretty economically than they Ought to have 3
6. Over the past few years, -> blacks have gotten quite economically than they Ought to have 3
7. Ove

True

## Q7

In [44]:
#!export
#QuestionID from the questions_to_be_translated data frame
question_id = 281

#index
kw_index_original = ["Significant", "Vital", "Crucial", "Essential", "Pivotal", "Fundamental", "Key", "Critical", "Paramount", "Imperative"] # add keywords for original question
kw_index_oposite = ["Insignificant", "Unimportant", "Trivial", "Irrelevant", "Inconsequential", "Minor", "Negligible", "Petty", "Nonessential", "Dispensable"] # add keywords for oposite question
dict_index = dict(dict_same_weight(1,kw_index_original), **dict_same_weight(-1,kw_index_oposite))

#scale
q7d:SCALE={
            "Worst":1,
            "Very Poor":2,
            "Poor":3,
            "Below Average":4,
            "Average":5,
            "Above average":6,
            "Good":7,
            "Very Good":8,
            "Excellent":9,
            "Best":10,
        }
dict_scale = q7d # change if you want a diffrent scale

#optional context dimensions
kw_some_context = []
dict_some_context = dict_same_weight(1,kw_some_context)


class PVQ26(QMNLI):
  """
  Write here the original question

  MNLI Query example:
        premise= write here an example of a premise ,
        hypothesis= write here an example of a hypothesis,
  """
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="I think it is {scale} to do things in traditional ways,",
        hypothesis_template="It's {index} to me to keep up with customs i learned",
        dimensions={
            "index":dict_index,
            "scale":dict_scale,
        },
        descriptor = {"Questionnair":"Portrait Values Questionnarie (PVQ)",
                      "Factor":"Hedonism",
                      "Ordinal":26,
                      "Original":'He thinks it is best to do things in traditional ways. It is important to him to keep up the customs he has learned',
        },
        **kwargs,
    )

#instantiation
q = PVQ26()
#running the question with some model stored in mnli
q = q.run(mnli)
#printing the model answers. You may use filters to report the question results in different contexts.
index = ["index"]
scale = "scale"
q.report(index=index,scale=scale,filters={"unfiltered":{}})
#Add the question with its index and scale to the list of all questions.
all_my_questions[(question_id, student_id)] = (q, index, scale)
print_permutations(q)
check_question_not_default(q, q.get_class_name())

Query time: 16.54053497314453
Mean score unfiltered: 268.189850139006
Internal consistency (silhouette, correlation) for unfiltered: 0.8585067784080529
Internal consistency (Calinski&Harabasz)  for unfiltered: 85.98782986276139
Internal consistency (Davies&Bouldin) for unfiltered: 0.39751015633966097


index = ['index']
{'scale', 'index'} {'scale'} {'index'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


scale,Worst,Very Poor,Poor,Below Average,Average,Above average,Good,Very Good,Excellent,Best
index,,,,,,,,,,
Trivial,0.7157,0.9042,0.9057,0.6464,0.1024,0.0096,0.0001,0.0001,0.0000,0.0001
Dispensable,0.9414,0.8678,0.8368,0.6688,0.0931,0.1291,0.0004,0.0005,0.0002,0.0008
Petty,0.9468,0.9855,0.9838,0.8421,0.0016,0.0018,0.0001,0.0000,0.0000,0.0005
Inconsequential,0.9357,0.9509,0.9191,0.8027,0.0005,0.0003,0.0001,0.0000,0.0000,0.0010
Insignificant,0.7091,0.9969,0.9960,0.9784,0.3909,0.0022,0.0004,0.0002,0.0001,0.0006
Irrelevant,0.8176,0.8990,0.9319,0.6955,0.0123,0.0017,0.0001,0.0001,0.0001,0.0002
Unimportant,0.9015,0.9272,0.9222,0.7792,0.0003,0.0002,0.0000,0.0000,0.0000,0.0001
Minor,0.9792,0.9994,0.9995,0.9943,0.9928,0.9459,0.0118,0.0016,0.0010,0.3459
Negligible,0.9630,0.9888,0.9966,0.9166,0.4772,0.3884,0.0001,0.0001,0.0000,0.0003


{'Questionnair': 'Portrait Values Questionnarie (PVQ)', 'Factor': 'Hedonism', 'Ordinal': 26, 'Original': 'He thinks it is best to do things in traditional ways. It is important to him to keep up the customs he has learned', 'scale': 'intensifier', 'index': "['scale', 'index']", 'query': "I think it is {scale} to do things in traditional ways,->It's {index} to me to keep up with customs i learned"}
0. I think it is Worst to do things in traditional ways, -> It's Significant to me to keep up with customs i learned 1
1. I think it is Very Poor to do things in traditional ways, -> It's Significant to me to keep up with customs i learned 2
2. I think it is Poor to do things in traditional ways, -> It's Significant to me to keep up with customs i learned 3
3. I think it is Below Average to do things in traditional ways, -> It's Significant to me to keep up with customs i learned 4
4. I think it is Average to do things in traditional ways, -> It's Significant to me to keep up with customs i l

True

## Q8

In [45]:
#!export
#QuestionID from the questions_to_be_translated data frame
question_id = 237

#index
kw_index_original = synonyms_tend_to = ["Lean", "Incline", "Tilt", "Veer", "Lean toward", "Be likely", "Be disposed", "Have a propensity"]
 # add keywords for original question
kw_index_oposite = ["Resist", "Oppose", "Refrain", "Avoid", "Disinclined", "Abstain", "Reject", "Dislike", "Disagree"] # add keywords for oposite question
dict_index = dict(dict_same_weight(1,kw_index_original), **dict_same_weight(-1,kw_index_oposite))

q8d:SCALE={
            "Don't Want To":1,
            "Prefer Not To":2,
            "Am Not Willing To":3,
            "Am Willing To":4,
            "Desire To":5,
            "Have To":6
        }
#scale
dict_scale = q8d # change if you want a diffrent scale

#optional context dimensions
kw_some_context = []
dict_some_context = dict_same_weight(1,kw_some_context)


class SOCQ21(QMNLI):
  """
  Write here the original question

  MNLI Query example:
        premise= write here an example of a premise ,
        hypothesis= write here an example of a hypothesis,
  """
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="Over the last years,",
        hypothesis_template="I {index} feeling inside i {scale} feel",
        dimensions={
            "index":dict_index,
            "scale":dict_scale,
        },
        descriptor = {"Questionnair":"SOC",
                      "Factor":"Comprehensibility",
                      "Ordinal":21,
                      "Original":'Does it happen that you have feelings inside you would rather not feel?',
        },
        **kwargs,
    )

#instantiation
q = SOCQ21()
#running the question with some model stored in mnli
q = q.run(mnli)
#printing the model answers. You may use filters to report the question results in different contexts.
index = ["index"]
scale = "scale"
q.report(index=index,scale=scale,filters={"unfiltered":{}})
#Add the question with its index and scale to the list of all questions.
all_my_questions[(question_id, student_id)] = (q, index, scale)
print_permutations(q)
check_question_not_default(q, q.get_class_name())

Query time: 6.5475921630859375
Mean score unfiltered: 10.392142846714705
Internal consistency (silhouette, correlation) for unfiltered: 0.4050832388094953
Internal consistency (Calinski&Harabasz)  for unfiltered: 6.512741199121018
Internal consistency (Davies&Bouldin) for unfiltered: 1.2475176656308382


index = ['index']
{'scale', 'index'} {'scale'} {'index'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


scale,Don't Want To,Prefer Not To,Am Not Willing To,Am Willing To,Desire To,Have To
index,,,,,,
Abstain,0.4900,0.2789,0.1729,0.6221,0.5052,0.5236
Avoid,0.4830,0.3903,0.4257,0.3683,0.4222,0.4033
Resist,0.4309,0.3171,0.3634,0.3692,0.4692,0.4980
Reject,0.6325,0.4175,0.5377,0.4713,0.5366,0.6075
Disagree,0.5173,0.4903,0.5815,0.5396,0.6426,0.6299
Disinclined,0.5243,0.3865,0.4243,0.4842,0.4381,0.4666
Dislike,0.4512,0.3630,0.4914,0.5391,0.5522,0.5452
Refrain,0.1793,0.2471,0.0814,0.0265,0.0060,0.1809
Oppose,0.5175,0.4019,0.5463,0.4582,0.5492,0.5556


{'Questionnair': 'SOC', 'Factor': 'Comprehensibility', 'Ordinal': 21, 'Original': 'Does it happen that you have feelings inside you would rather not feel?', 'scale': 'intensifier', 'index': "['scale', 'index']", 'query': 'Over the last years,->I {index} feeling inside i {scale} feel'}
0. Over the last years, -> I Lean feeling inside i Don't Want To feel 1
1. Over the last years, -> I Lean feeling inside i Prefer Not To feel 2
2. Over the last years, -> I Lean feeling inside i Am Not Willing To feel 3
3. Over the last years, -> I Lean feeling inside i Am Willing To feel 4
4. Over the last years, -> I Lean feeling inside i Desire To feel 5
5. Over the last years, -> I Lean feeling inside i Have To feel 6
6. Over the last years, -> I Incline feeling inside i Don't Want To feel 1
7. Over the last years, -> I Incline feeling inside i Prefer Not To feel 2
8. Over the last years, -> I Incline feeling inside i Am Not Willing To feel 3
9. Over the last years, -> I Incline feeling inside i Am Wi

True

## Q9

In [46]:
#!export
#QuestionID from the questions_to_be_translated data frame
question_id = 338

#index
kw_index_original = ["Confident", "Fulfilled", "Satisfied", "Elated", "Esteemed", "Honored", "Self-assured"] # add keywords for original question
kw_index_oposite = ["Ashamed", "Humiliated", "Shameful", "Abashed", "Dishonored"] # add keywords for oposite question
dict_index = dict(dict_same_weight(1,kw_index_original), **dict_same_weight(-1,kw_index_oposite))
q9d:SCALE = {
    'a bit':1,
    'slightly':1,
    'somewhat':1,
    'pretty':3,
    'quite':3,
    'very':4,
    'completely':5,
    'totally':5
}
#scale
dict_scale = q9d # change if you want a diffrent scale

#optional context dimensions
kw_some_context = []
dict_some_context = dict_same_weight(1,kw_some_context)


class CCRAMQ4(QMNLI):
  """
  Write here the original question

  MNLI Query example:
        premise= write here an example of a premise ,
        hypothesis= write here an example of a hypothesis,
  """
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="In my community,",
        hypothesis_template="I'm {scale} {index} to tell others where I live",
        dimensions={
            "index":dict_index,
            "scale":dict_scale,
        },
        descriptor = {"Questionnair":"CCRAM",
                      "Factor":"Place Attachment",
                      "Ordinal":4,
                      "Original":'In my community, I am proud to tell others where I live',
        },
        **kwargs,
    )

#instantiation
q = CCRAMQ4()
#running the question with some model stored in mnli
q = q.run(mnli)
#printing the model answers. You may use filters to report the question results in different contexts.
index = ["index"]
scale = "scale"
q.report(index=index,scale=scale,filters={"unfiltered":{}})
#Add the question with its index and scale to the list of all questions.
all_my_questions[(question_id, student_id)] = (q, index, scale)
print_permutations(q)
check_question_not_default(q, q.get_class_name())

Query time: 7.8897926807403564
Mean score unfiltered: 29.982839867428993
Internal consistency (silhouette, correlation) for unfiltered: 0.4431095611923154
Internal consistency (Calinski&Harabasz)  for unfiltered: 5.037591015244303
Internal consistency (Davies&Bouldin) for unfiltered: 1.2729818383789828


index = ['index']
{'scale', 'index'} {'scale'} {'index'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


scale,a bit,slightly,somewhat,pretty,quite,very,completely,totally
index,,,,,,,,
Abashed,0.7149,0.6018,0.6856,0.2099,0.2815,0.0974,0.0027,0.0017
Ashamed,0.3861,0.3686,0.4524,0.1336,0.1164,0.0485,0.0396,0.0359
Dishonored,0.7842,0.6672,0.7673,0.2225,0.1845,0.0788,0.0002,0.0001
Humiliated,0.2421,0.1861,0.2673,0.0602,0.0301,0.0045,0.0014,0.0012
Shameful,0.6819,0.5375,0.6462,0.3963,0.3608,0.2156,0.0899,0.1109
Confident,0.7270,0.5630,0.6785,0.4547,0.4699,0.3848,0.0095,0.0115
Elated,0.8439,0.7082,0.8119,0.4542,0.4726,0.3588,0.0731,0.0731
Esteemed,0.8529,0.7386,0.8336,0.6371,0.6525,0.6141,0.2750,0.2663
Fulfilled,0.6648,0.5741,0.6649,0.2974,0.4038,0.3394,0.0969,0.1035


{'Questionnair': 'CCRAM', 'Factor': 'Place Attachment', 'Ordinal': 4, 'Original': 'In my community, I am proud to tell others where I live', 'scale': 'intensifier', 'index': "['scale', 'index']", 'query': "In my community,->I'm {scale} {index} to tell others where I live"}
0. In my community, -> I'm a bit Confident to tell others where I live 1
1. In my community, -> I'm slightly Confident to tell others where I live 1
2. In my community, -> I'm somewhat Confident to tell others where I live 1
3. In my community, -> I'm pretty Confident to tell others where I live 3
4. In my community, -> I'm quite Confident to tell others where I live 3
5. In my community, -> I'm very Confident to tell others where I live 4
6. In my community, -> I'm completely Confident to tell others where I live 5
7. In my community, -> I'm totally Confident to tell others where I live 5
8. In my community, -> I'm a bit Fulfilled to tell others where I live 1
9. In my community, -> I'm slightly Fulfilled to tell ot

True

## Q10

In [47]:
#!export
#QuestionID from the questions_to_be_translated data frame
question_id = 291

#index
kw_index_original =  ["Significant", "Vital", "Crucial", "Essential", "Pivotal", "Fundamental", "Key", "Critical", "Paramount", "Imperative"] # add keywords for original question
kw_index_oposite = ["Insignificant", "Unimportant", "Trivial", "Irrelevant", "Inconsequential", "Minor", "Negligible", "Petty", "Nonessential", "Dispensable"] # add keywords for oposite question
dict_index = dict(dict_same_weight(1,kw_index_original), **dict_same_weight(-1,kw_index_oposite))
q10d:SCALE = {
   "Not at all":1,
   "Not so":2,
   "Almost not":3,
   "Very":4,
   "Most":5
}
#scale
dict_scale = q10d # change if you want a diffrent scale

#optional context dimensions
kw_some_context = []
dict_some_context = dict_same_weight(1,kw_some_context)


class PVQ36(QMNLI):
  """
  Write here the original question

  MNLI Query example:
        premise= write here an example of a premise ,
        hypothesis= write here an example of a hypothesis,
  """
  def __init__(self, **kwargs):
    super().__init__(
        premise_template="In my opinion,",
        hypothesis_template="having a stable government is {scale} {index}, I'm concerned that the social order be protected",
        dimensions={
            "index":dict_index,
            "scale":dict_scale,
        },
        descriptor = {"Questionnair":"Portrait Values Questionnarie (PVQ)",
                      "Factor":"Conformity",
                      "Ordinal":36,
                      "Original":'having a stable government is important to him. He is concerned that the social order be protected',
        },
        **kwargs,
    )

#instantiation
q = PVQ36()
#running the question with some model stored in mnli
q = q.run(mnli)
#printing the model answers. You may use filters to report the question results in different contexts.
index = ["index"]
scale = "scale"
q.report(index=index,scale=scale,filters={"unfiltered":{}})
#Add the question with its index and scale to the list of all questions.
all_my_questions[(question_id, student_id)] = (q, index, scale)
print_permutations(q)
check_question_not_default(q, q.get_class_name())

Query time: 11.545456886291504
Mean score unfiltered: -1.083050787448883
Internal consistency (silhouette, correlation) for unfiltered: 0.0
Internal consistency (Calinski&Harabasz)  for unfiltered: 3.115519925055018
Internal consistency (Davies&Bouldin) for unfiltered: 1.348009438933145


index = ['index']
{'scale', 'index'} {'scale'} {'index'}
set()


<ipython-input-13-1bc4b0d3db87>:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s = s.set_precision(4)


scale,Not at all,Not so,Almost not,Very,Most
index,,,,,
Trivial,0.6287,0.6474,0.6362,0.6287,0.6287
Dispensable,0.6287,0.6287,0.6287,0.6287,0.6287
Petty,0.6287,0.6287,0.6343,0.6287,0.6287
Inconsequential,0.6287,0.6287,0.6652,0.6287,0.6287
Insignificant,0.6298,0.6891,0.6774,0.6287,0.6287
Irrelevant,0.6287,0.6287,0.6287,0.6287,0.6287
Unimportant,0.6287,0.6287,0.6287,0.6287,0.6287
Minor,0.6524,0.7155,0.7625,0.6287,0.6287
Negligible,0.6287,0.6287,0.6287,0.6287,0.6287


{'Questionnair': 'Portrait Values Questionnarie (PVQ)', 'Factor': 'Conformity', 'Ordinal': 36, 'Original': 'having a stable government is important to him. He is concerned that the social order be protected', 'scale': 'intensifier', 'index': "['scale', 'index']", 'query': "In my opinion,->having a stable government is {scale} {index}, I'm concerned that the social order be protected"}
0. In my opinion, -> having a stable government is Not at all Significant, I'm concerned that the social order be protected 1
1. In my opinion, -> having a stable government is Not so Significant, I'm concerned that the social order be protected 2
2. In my opinion, -> having a stable government is Almost not Significant, I'm concerned that the social order be protected 3
3. In my opinion, -> having a stable government is Very Significant, I'm concerned that the social order be protected 4
4. In my opinion, -> having a stable government is Most Significant, I'm concerned that the social order be protected 

True

# Testing you assignment

Here you can test your questions using linguestics acceptability test

In [ ]:
if len(all_my_questions) < 10:
  print(f'You only added {len(all_my_questions)} questions')

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for (question_id, student_id), (q, index, scale) in all_my_questions.items():
      question_name = q.get_class_name()
      linguistic_df = linguistic_acceptabilities(q, question_id, index=index, scale=scale,question_name=question_name, student_id=student_id)
      row = linguistic_df[['cola_score','silhouette_score']].mean(axis=0)
      score = dict(row)
      score['semantic_similarity'] = linguistic_df['semantic_similarity'].quantile(0.75)
      print('Question:', question_name, 'results:', score)
      print('Is question is not default?:', check_question_not_default(q, question_name))

# FAQ



**How will the assignment be graded?**

The grading criteria for this assignment will be based on three equal-weighted parameters:

1.	**Linguistic Acceptability (LA or CoLA)**
>LA refers to the level of acceptability of sentences in English. It assesses how well a sentence adheres to the English rules and conventions, including syntax, grammar, and spelling. LA is influenced by factors such as sentence structure, word order, verb agreement, tense consistency, and proper usage of punctuation and capitalization. You can find more information about CoLA here: https://nyu-mll.github.io/CoLA/

2.	**Silhouette**
>The Silhouette score is a measure employed to evaluate the quality of clustering results. It quantifies how well each data point fits within its assigned cluster. The Silhouette score ranges from -1 to +1, where a score close to +1 suggests a clear separation between clusters. This indicates that each data point within a cluster is well-matched to its own cluster and poorly matched to neighboring clusters.
You can find more information about Silhouette here: https://www.sciencedirect.com/topics/computer-science/Silhouette-coefficient

3.	**Semantic Similarity (SS) Between the Translated Questionnaire and the Translated Item**
>SS refers to the degree of likeness or resemblance between two pieces of text in terms of meaning. In the context of this assignment, SS is measured individually for all phrases of a translated item, and the grade is determined as the average of the highest quartile of measurements. We use SS to assess the extent to which the permutations created by the translation represent the construct that the original item aims to measure (content validity).
You can find more information about SS here: https://paperswithcode.com/task/semantic-similarity


This notebook calculates all parameters, for your convenience.

**Do the different parameters affect each other?**

No. They are all related to the phrases that are created by the translation but don’t affect each other directly. Improving one parameter does not necessarily affect another.


**Which numerical scale should I use?**

For each item, you can use the same numerical scale as is in it. However, you can replace the numerical scale so it will be a 1 to 5 Likert scale.


**What can I do if the translated item does not have a context/term that can serve as a premise?**

Each translation must have a premise. In absence of an explicit premise in the item you should create a nonspecific premise, such as “Generally”, “In most cases”, “When I think about…”.


**Do I have to use the exact same wording used in the item that I’m translating?**

No. You can use any wording you like.


**I got different colors on the table representing the probability of the phrases, but the values in the cells seem as if they all have the same number. Is that OK?**

Yes. The values in the table’s cells are given to fifth place after the decimal dot. The situation described in this question means that the differences between the probabilities are smaller than 10^(-5).


**In what format should I submit my assignment?**

An IPYNB file that its name is your 9-digit ID number seperated with an underscore from your course number.ipynb, compressed to a ZIP archive that its name is your 9-digit ID number seperated with an underscore from your course number.zip.
e.g., if your ID number is 012345678 and the course number is 372148030120231, than the name of the IPYNB file should be 012345678_372148030120231.ipynb, and that file should be compressed in a zip archive named 012345678_372148030120231.zip.


**How many indexes and intensifiers terms are required for each translated item?**

At least 4 terms for indexes (2 original and 2 opposite) and 3 terms for intensifiers.


**My item has terms that I am not sure about their meanings. What should I do?**

You can use online tools such as ChatGPT and [THESAURUS](https://https://www.thesaurus.com/) to better understand the meaning of the terms, and the purpose of the item. You can also ask the assignment support personnel (Idan).


**Is there any way that I can easily get a nice bonus on this assignment’s score?**

I’m glad you asked that! Before the final scores will be calculated, we will ask you to grant us permission to use the products of your assignment for research purposes. If you choose to do so, you will sign a waiver and relinquish all the rights of your assignment. In return you will receive a 5 points bonus points to your overall score on this assignment.


# References:
*   Beullens Koen, Geert Loosveldt, Caroline Vandenplas, and Ineke Stoop. Response
rates in the European social survey: Increasing, decreasing, or a matter of fieldwork efforts? Survey methods: Insights from the field, pages 1–12, 2018

*   Einola, K., & Alvesson, M. (2021). Behind the Numbers: Questioning Questionnaires. Journal of Management Inquiry, 30(1), 102–114. https://doi.org/10.1177/1056492620938139

*   Lu Ann Aday and Llewellyn J Cornelius. Designing and conducting health surveys: a comprehensive guide. John Wiley & Sons, 2006.

*   Nikhil Garg, Londa Schiebinger, Dan Jurafsky, and James Zou. Word embeddings quantify 100 years of gender and ethnic stereotypes. Proceedings of the National Academy of Sciences, 115(16):E3635–E3644, 2018.
